In [37]:
import numpy as np
import pandas as pd
import urllib.request, json 

pd.set_option('display.max_columns', None)

In [38]:
# df = pd.read_csv('./us-states.csv')
# df = df[df.date == max(df.date)]
# df = df[(df.state != 'Guam') & (df.state != 'Puerto Rico') & (df.state != 'Virgin Islands')]
# df.drop(['fips', 'date'], axis=1, inplace=True)
# df.columns = ['State', 'Infected', 'Deaths']
# df.head(3)

### Live Infected, Deaths, COVID19 Testing data
(could maybe use this data for number of infections and deaths per state)  
http://coronavirusapi.com/states.csv

In [39]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [40]:
df = pd.read_csv('http://coronavirusapi.com/states.csv')
df['name,'] = df['name,'].apply(lambda r: states[r])
df.columns = ['State', 'Tested', 'Infected', 'Deaths']
df.head()

,State,Tested,Infected,Deaths
0,Alaska,5022,143,3.0
1,Alabama,8619,1179,32.0
2,Arkansas,8461,625,10.0
3,Arizona,21058,1413,29.0
4,California,90100,8155,171.0


### Total Population

In [41]:
df_pop = pd.read_csv('./predictor_data/population.csv')
df_pop = df_pop[['State', 'Pop', 'density']]
df_pop.columns =['State', 'Population', 'Pop Density']

In [42]:
df = df.merge(df_pop, on='State', how='inner')

### Gini coefficient

In [43]:
df = df.merge(pd.read_csv('./predictor_data/gini.csv'), on='State', how='inner')

### ICU beds

In [44]:
cols_of_interest = ['state', 'all_icu']
df_icu = pd.read_excel('./predictor_data/icu_beds.xlsx', usecols=cols_of_interest)
df_icu = df_icu.groupby('state').sum().reset_index()
df_icu.columns = ['State', 'ICU Beds']

In [45]:
df = df.merge(df_icu, on='State', how='inner')

### Income per capita

In [46]:
df = df.merge(pd.read_csv('./predictor_data/income_per_capita.csv'), on='State', how='inner')
df.shape

(51, 9)

### GDP

In [47]:
df = df.merge(pd.read_csv('./predictor_data/state_gdp.csv'), on='State', how='inner')

### Unemployment

In [48]:
df = df.merge(pd.read_csv('./predictor_data/unemployment.csv'), on='State', how='inner')

### Sex

In [49]:
df_sex = pd.read_csv('./predictor_data/sex.csv')
df_sex['Sex Ratio'] = df_sex.Male / df_sex.Female 
df_sex = df_sex[['Location', 'Sex Ratio']]
df_sex.columns = ['State', 'Sex Ratio']

In [50]:
df = df.merge(df_sex, on='State', how='inner')

### Smoking

In [51]:
df_smoke = pd.read_csv('./predictor_data/smoking.csv')
df_smoke.columns = ['State', 'Smoking Rate']

In [52]:
df = df.merge(df_smoke, on='State', how='inner')

### Flu and Pneumonia Death Rate

In [53]:
df_flu = pd.read_csv('./predictor_data/Influenza_Pneumonia Mortality by State.csv')
df_flu = df_flu[df_flu.YEAR == 2018]
df_flu = df_flu[['STATE', 'RATE']]
df_flu.columns = ['State', 'Flu Deaths']
df_flu['State'] = df_flu['State'].apply(lambda r: states[r])
dc_df = pd.DataFrame([['District of Columbia', 11.2]], columns=['State', 'Flu Deaths']) # https://www.kff.org/other/state-indicator/influenza-and-pneumonia-death-rate/
df_flu = df_flu.append(dc_df, ignore_index=True)

In [54]:
df = df.merge(df_flu, on='State', how='inner')

### Chronic Lower Respiratory Disease Death Rate

In [55]:
df_resp = pd.read_csv('./predictor_data/Chronic Lower Respiratory Disease Mortality by State.csv')
df_resp = df_resp[df_resp.YEAR == 2018]
df_resp = df_resp[['STATE', 'RATE']]
df_resp.columns = ['State', 'Respiratory Deaths']
df_resp['State'] = df_resp['State'].apply(lambda r: states[r])
dc_df = pd.DataFrame([['District of Columbia', 19.6]], columns=['State', 'Respiratory Deaths']) # https://www.cdc.gov/nchs/pressroom/states/dc/dc.htm
df_resp = df_resp.append(dc_df, ignore_index=True)

In [56]:
df = df.merge(df_resp, on='State', how='inner')

### Doctors

In [57]:
df_doc = pd.read_csv('./predictor_data/doctors.csv')
df_doc = df_doc[['Location', 'Total']]
df_doc.columns = ['State', 'Physicians']

In [58]:
df = df.merge(df_doc, on='State', how='inner')

### Number Hospitals

In [59]:
df_hosp = pd.read_csv('./predictor_data/hospitals.csv')
df_hosp.columns = ['State', 'Hospitals']

In [60]:
df = df.merge(df_hosp, on='State', how='inner')

### Health Spending per capita

In [61]:
df_spend = pd.read_csv('./predictor_data/health_spending.csv')
df_spend.columns = ['State', 'Health Spending']

In [62]:
df = df.merge(df_spend, on='State', how='inner')

### Pollution

In [63]:
pollution_url = 'https://www.americashealthrankings.org/api/v1/measures/metrics/201?measure_id=147'
with urllib.request.urlopen(pollution_url) as url:
    data = json.loads(url.read().decode())
pollution = [{'State': states[i['StateCode']], 'Pollution':float(i['Value'])} for i in data['items'] if i['StateCode'] != 'ALL']
df_pol = pd.DataFrame(pollution)

In [64]:
df = df.merge(df_pol, on='State', how='inner')

### Airport

In [65]:
df_air = pd.read_csv('./predictor_data/airports.csv')
df_air['State'].value_counts()
df_air = df_air['State'].value_counts().rename_axis('State').reset_index(name='Med-Large Airports')
df_air['State'] = df_air['State'].apply(lambda r: states[r])
df_air.shape

(34, 2)

In [66]:
df = df.merge(df_air, on='State', how='left')
df = df.fillna(0)

### Avg Temperature

In [67]:
df = df.merge(pd.read_csv('./predictor_data/temperature.csv'), on='State', how='inner')

### Urbanization

In [68]:
df = df.merge(pd.read_csv('./predictor_data/urbanization.csv'), on='State', how='inner')

### Age Groups

In [69]:
df_age = pd.read_csv('./predictor_data/age.csv')
df_age['State'] = df_age['Location']
df_age['Age 0-25'] = df_age['Children 0-18'] + df_age['Adults 19-25']
df_age['Age 26-54'] = df_age['Adults 26-34'] + df_age['Adults 35-54']
df_age['Age 55+'] = df_age['Adults 55-64'] + df_age['65+']
df_age.drop(['Location', 'Children 0-18', 'Adults 19-25', 'Adults 26-34', 'Adults 35-54', 'Total', 'Footnotes', 'Adults 55-64', '65+'], axis=1, inplace=True)

In [70]:
df = df.merge(df_age, on='State', how='inner')

### School Closure Dates

In [71]:
df = df.merge(pd.read_csv('./predictor_data/school_closures.csv'), on='State', how='inner')

## Print!

In [72]:
df.to_csv('COVID19_state.csv', index=False)
df.head()

,State,Tested,Infected,Deaths,Population,Pop Density,Gini,ICU Beds,Income,GDP,Unemployment,Sex Ratio,Smoking Rate,Flu Deaths,Respiratory Deaths,Physicians,Hospitals,Health Spending,Pollution,Med-Large Airports,Temperature,Urban,Age 0-25,Age 26-54,Age 55+,School Closure Date
0,Alaska,5022,143,3.0,734002,1.2863,0.4081,119,59687,73205,5.8,1.054688,21.0,12.1,35.3,1900,21,11064,6.4,1.0,26.6,66.0,0.36,0.39,0.25,03/19/20
1,Alabama,8619,1179,32.0,4908621,96.9221,0.4847,1533,42334,45219,2.7,0.930145,20.9,21.4,58.0,12205,101,7281,8.1,1.0,62.8,59.0,0.33,0.37,0.31,03/16/20
2,Arkansas,8461,625,10.0,3038999,58.4030,0.4719,732,42566,42454,3.5,0.956630,22.3,18.0,61.7,7150,88,7408,7.1,0.0,60.4,56.2,0.34,0.37,0.30,03/17/20
3,Arizona,21058,1413,29.0,7378494,64.9550,0.4713,1559,43650,48055,4.5,0.966965,15.6,12.4,41.2,17806,83,6452,9.7,1.0,60.3,89.8,0.33,0.36,0.30,03/16/20
4,California,90100,8155,171.0,39937489,256.3727,0.4899,7338,62586,74205,3.9,0.975113,11.3,15.6,30.9,112906,359,7549,12.8,9.0,59.4,95.0,0.33,0.40,0.26,03/19/20
